In [6]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df["start time"] = pd.to_datetime(df["start time"], format="%H:%M:%S")
df["end time"] = pd.to_datetime(df["end time"], format="%H:%M:%S")

# Trek ze van elkaar af → dit geeft een timedelta
df["duur"] = df["end time"] - df["start time"]

df.head(5)

NameError: name 'pd' is not defined

In [ ]:
df_m["min_travel_time"] = pd.to_timedelta(df_m["min_travel_time"], unit = "m")
df_m["max_travel_time"] = pd.to_timedelta(df_m["max_travel_time"], unit = "m")
df_m.head(5)


NameError: name 'df_m' is not defined

In [3]:
import pandas as pd

# Inlezen
planning = pd.read_excel("Bus Planning.xlsx")
matrix = pd.read_excel("DistanceMatrix.xlsx")

# Alles naar string zetten (veilig)
planning["start location"] = planning["start location"].astype(str)
planning["end location"] = planning["end location"].astype(str)
planning["line"] = planning["line"].astype(str)

matrix["start"] = matrix["start"].astype(str)
matrix["end"] = matrix["end"].astype(str)
matrix["line"] = matrix["line"].astype(str)

# Merge op drie kolommen tegelijk
matched = planning.merge(
    matrix,
    left_on=["start location", "end location", "line"],
    right_on=["start", "end", "line"],
    how="inner"
)

matched

,start location,end location,start time,end time,activity,line,energy consumption,bus,start,end,min_travel_time,max_travel_time,distance_m
0,ehvgar,ehvbst,05:03:00,05:07:00,material trip,nan,1.9800,1,ehvgar,ehvbst,4,4,1650
1,ehvbst,ehvapt,05:07:00,05:31:00,service trip,401.0,10.8036,1,ehvbst,ehvapt,22,24,9003
2,ehvapt,ehvbst,06:04:00,06:29:00,service trip,401.0,10.8600,1,ehvapt,ehvbst,22,25,9050
3,ehvbst,ehvapt,06:31:00,06:55:00,service trip,401.0,10.8036,1,ehvbst,ehvapt,22,24,9003
4,ehvapt,ehvbst,06:56:00,07:21:00,service trip,401.0,10.8600,1,ehvapt,ehvbst,22,25,9050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,ehvbst,ehvapt,17:06:00,17:30:00,service trip,400.0,12.8496,19,ehvbst,ehvapt,21,23,10708
574,ehvapt,ehvgar,17:30:00,17:50:00,material trip,nan,10.8000,19,ehvapt,ehvgar,20,20,9000
575,ehvgar,ehvbst,14:02:00,14:06:00,material trip,nan,1.9800,20,ehvgar,ehvbst,4,4,1650
576,ehvbst,ehvapt,14:06:00,14:30:00,service trip,400.0,12.8496,20,ehvbst,ehvapt,21,23,10708


In [ ]:
matched["min_travel_time"] = pd.to_timedelta(matched["min_travel_time"], unit = "m")
matched["max_travel_time"] = pd.to_timedelta(matched["max_travel_time"], unit = "m")
matched["start time"] = pd.to_datetime(matched["start time"], format="%H:%M:%S")
matched["end time"] = pd.to_datetime(matched["end time"], format="%H:%M:%S")
matched["duur"] = matched["end time"] - matched["start time"]

NameError: name 'matched' is not defined

In [ ]:
matched.head()
df = matched.drop(columns=["start", "end"])
df.head(50)

start location end location start time  end time       activity   line  \
0         ehvgar       ehvbst   05:03:00  05:07:00  material trip    nan   
1         ehvbst       ehvapt   05:07:00  05:31:00   service trip  401.0   
2         ehvapt       ehvbst   06:04:00  06:29:00   service trip  401.0   
3         ehvbst       ehvapt   06:31:00  06:55:00   service trip  401.0   
4         ehvapt       ehvbst   06:56:00  07:21:00   service trip  401.0   

   energy consumption  bus  min_travel_time  max_travel_time  distance_m  
0              1.9800    1                4                4        1650  
1             10.8036    1               22               24        9003  
2             10.8600    1               22               25        9050  
3             10.8036    1               22               24        9003  
4             10.8600    1               22               25        9050

In [ ]:
def travel_time(df):
    n = len(df)
    invalid = []
    for i in range (n):
        if df["duur"][i] > df["max_travel_time"][i] or df["duur"][i] < df["min_travel_time"][i]:
            invalid.append(i)
    return invalid

In [ ]:
def invalid_start_time(df):
    n = len(df)
    invalid2 = []
    for i in range (n):
        if df["duur"][i] < pd.Timedelta(0):
            invalid2.append(i)
    print(f'let op, controleer de volgende rij(en) {invalid2} of deze snachts rijden. Als dit niet zo is dan vertrekt de bus eerder dan dat het aankomt, dus klopt dat niet')
    return invalid2

invalid_start_time(df)

let op, controleer de volgende rij(en) [44, 90] of deze snachts rijden. Als dit niet zo is dan vertrekt de bus eerder dan dat het aankomt, dus klopt dat niet


[44, 90]

In [ ]:
def dubbele_bus(df):
    n = len(df)
    invalid3 = []
    for i in range (n-1):
        if df["bus"][i] == df["bus"][i+1]:
            if df["end time"][i] > df["start time"][i + 1]:
                invalid3.append((i, i+1))
    print(f'let op, controleer de volgende rij(en) {invalid3} of deze snachts rijden. Als dit niet zo is dan vertrekt de bus eerder dan dat het aankomt, dus klopt dat niet')
    return invalid3

dubbele_bus(df)

let op, controleer de volgende rij(en) [(133, 134)] of deze snachts rijden. Als dit niet zo is dan vertrekt de bus eerder dan dat het aankomt, dus klopt dat niet


[(133, 134)]

In [ ]:
df.iloc[51]

NameError: name 'df' is not defined

In [5]:
len(planning)
planning.tail()

,start location,end location,start time,end time,activity,line,energy consumption,bus
715,ehvbst,ehvapt,17:06:00,17:30:00,service trip,400.0,12.8496,19
716,ehvapt,ehvgar,17:30:00,17:50:00,material trip,nan,10.8000,19
717,ehvgar,ehvbst,14:02:00,14:06:00,material trip,nan,1.9800,20
718,ehvbst,ehvapt,14:06:00,14:30:00,service trip,400.0,12.8496,20
719,ehvapt,ehvgar,14:30:00,14:50:00,material trip,nan,10.8000,20


In [11]:
len(planning[planning['activity']=='service trip'])
service = planning[planning['activity']=='service trip']
service.head()

,start location,end location,start time,end time,activity,line,energy consumption,bus
1,ehvbst,ehvapt,05:07:00,05:31:00,service trip,401.0,10.8036,1
3,ehvapt,ehvbst,06:04:00,06:29:00,service trip,401.0,10.8600,1
5,ehvbst,ehvapt,06:31:00,06:55:00,service trip,401.0,10.8036,1
7,ehvapt,ehvbst,06:56:00,07:21:00,service trip,401.0,10.8600,1
9,ehvbst,ehvapt,07:21:00,07:45:00,service trip,400.0,12.8496,1


In [9]:
df = pd.read_excel('Timetable.xlsx')
df.head()

,start,departure_time,end,line
0,ehvapt,06:04,ehvbst,401
1,ehvapt,06:34,ehvbst,401
2,ehvapt,06:56,ehvbst,401
3,ehvapt,07:11,ehvbst,401
4,ehvapt,07:26,ehvbst,401


In [3]:
import pandas as pd
df2 = pd.read_excel('DistanceMatrix.xlsx')
df2

,start,end,min_travel_time,max_travel_time,distance_m,line
0,ehvapt,ehvbst,21,23,10250,400.0
1,ehvbst,ehvapt,21,23,10708,400.0
2,ehvapt,ehvbst,22,25,9050,401.0
3,ehvbst,ehvapt,22,24,9003,401.0
4,ehvapt,ehvbst,20,20,8600,NaN
5,ehvbst,ehvapt,20,20,8600,NaN
6,ehvbst,ehvgar,4,4,1650,NaN
7,ehvgar,ehvbst,4,4,1650,NaN
8,ehvapt,ehvgar,20,20,9000,NaN
9,ehvgar,ehvapt,20,20,9000,NaN


In [15]:
import pandas as pd

# Zorg dat beide line-kolommen hetzelfde type hebben
df["line"] = df["line"].astype(str)
service["line"] = service["line"].astype(str)

# Merge opnieuw uitvoeren
merged = pd.merge(
    df,
    service,
    left_on=["line", "start", "end"],
    right_on=["line", "start location", "end location"],
    how="left",
    indicator=True
)

# Ritten die niet voorkomen in uitvoering
missing_trips = merged[merged["_merge"] == "left_only"]

print("Aantal missende ritten:", len(missing_trips))
print(missing_trips[["line", "start", "end", "departure_time"]])


Aantal missende ritten: 327
    line   start     end departure_time
0    401  ehvapt  ehvbst          06:04
1    401  ehvapt  ehvbst          06:34
2    401  ehvapt  ehvbst          06:56
3    401  ehvapt  ehvbst          07:11
4    401  ehvapt  ehvbst          07:26
..   ...     ...     ...            ...
322  400  ehvbst  ehvapt          18:52
323  400  ehvbst  ehvapt          18:54
324  400  ehvbst  ehvapt          19:07
325  400  ehvbst  ehvapt          19:22
326  400  ehvbst  ehvapt          19:37

[327 rows x 4 columns]


C:\Users\doris\AppData\Local\Temp\ipykernel_15672\3749099574.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  service["line"] = service["line"].astype(str)


In [ ]:
#df['activity']=='idle'
df

,start location,end location,start time,end time,activity,line,energy consumption,bus,min_travel_time,max_travel_time,distance_m
0,ehvgar,ehvbst,05:03:00,05:07:00,idle,nan,1.9800,1,4,4,1650
1,ehvbst,ehvapt,05:07:00,05:31:00,idle,401.0,10.8036,1,22,24,9003
2,ehvapt,ehvbst,06:04:00,06:29:00,idle,401.0,10.8600,1,22,25,9050
3,ehvbst,ehvapt,06:31:00,06:55:00,idle,401.0,10.8036,1,22,24,9003
4,ehvapt,ehvbst,06:56:00,07:21:00,idle,401.0,10.8600,1,22,25,9050
...,...,...,...,...,...,...,...,...,...,...,...
573,ehvbst,ehvapt,17:06:00,17:30:00,idle,400.0,12.8496,19,21,23,10708
574,ehvapt,ehvgar,17:30:00,17:50:00,idle,nan,10.8000,19,20,20,9000
575,ehvgar,ehvbst,14:02:00,14:06:00,idle,nan,1.9800,20,4,4,1650
576,ehvbst,ehvapt,14:06:00,14:30:00,idle,400.0,12.8496,20,21,23,10708
